In [2]:
import pandas as pd
import numpy as np
import time
import requests
import urllib.request
from urllib.parse import quote_plus
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import unicodedata
import time
import random

In [3]:
#!pip3 install selenium

     |████████████████████████████████| 904 kB 2.7 MB/s eta 0:00:01


In [17]:
#!pip3 install xlrd==1.2.0

In [2]:
dining_list = pd.read_csv('dining_listup_kakao3.csv')
dining_list

,dining_id,total_name,dining_name,branch_name,category,addr,source
0,0,카덴 연희본점,카덴,연희본점,일식,\n서울 서대문구 연희로 173 거화빌딩 1층\n(지번) 연희동 81-1\n,카카오
1,1,앤트러사이트 연희점,앤트러사이트,연희점,커피전문점,\n서울 서대문구 연희로 135\n(지번) 연희동 89-19\n,카카오
2,2,목란,목란,NaN,중식,\n서울 서대문구 연희로15길 21\n(지번) 연희동 132-28\n,카카오
3,3,콘하스 연희점,콘하스,연희점,카페,\n서울 서대문구 연희로27길 99\n(지번) 연희동 90-1\n,카카오
4,4,연경 연희점,연경,연희점,중화요리,\n서울 서대문구 연희로 145\n(지번) 연희동 86-15\n,카카오
...,...,...,...,...,...,...,...
472,473,히우카우,히우카우,NaN,태국음식,\n서울 서대문구 연세로4길 61 1층\n(지번) 창천동 5-32\n,카카오
473,474,연남서식당,연남서식당,NaN,갈비,\n서울 마포구 백범로2길 32\n(지번) 노고산동 109-69\n,카카오
474,475,비아메렝게,비아메렝게,NaN,"멕시칸,브라질",\n서울 서대문구 명물길 27-18 1층\n(지번) 창천동 45-9\n,카카오
475,476,포티드,포티드,NaN,카페,\n서울 서대문구 명물길 36-3 1층\n(지번) 창천동 5-58\n,카카오


In [3]:
def add_region(x) :
    if '신촌' in x :
        return x
    else :
        return '신촌'+ x
        

In [4]:
dining_list['new_dining_name'] = dining_list['dining_name'].map(lambda x: add_region(x))
dining_list = dining_list[['dining_id','new_dining_name','total_name','dining_name','branch_name','addr']]
dining_list

,dining_id,new_dining_name,total_name,dining_name,branch_name,addr
0,0,신촌카덴,카덴 연희본점,카덴,연희본점,\n서울 서대문구 연희로 173 거화빌딩 1층\n(지번) 연희동 81-1\n
1,1,신촌앤트러사이트,앤트러사이트 연희점,앤트러사이트,연희점,\n서울 서대문구 연희로 135\n(지번) 연희동 89-19\n
2,2,신촌목란,목란,목란,NaN,\n서울 서대문구 연희로15길 21\n(지번) 연희동 132-28\n
3,3,신촌콘하스,콘하스 연희점,콘하스,연희점,\n서울 서대문구 연희로27길 99\n(지번) 연희동 90-1\n
4,4,신촌연경,연경 연희점,연경,연희점,\n서울 서대문구 연희로 145\n(지번) 연희동 86-15\n
...,...,...,...,...,...,...
472,473,신촌히우카우,히우카우,히우카우,NaN,\n서울 서대문구 연세로4길 61 1층\n(지번) 창천동 5-32\n
473,474,신촌연남서식당,연남서식당,연남서식당,NaN,\n서울 마포구 백범로2길 32\n(지번) 노고산동 109-69\n
474,475,신촌비아메렝게,비아메렝게,비아메렝게,NaN,\n서울 서대문구 명물길 27-18 1층\n(지번) 창천동 45-9\n
475,476,신촌포티드,포티드,포티드,NaN,\n서울 서대문구 명물길 36-3 1층\n(지번) 창천동 5-58\n


In [12]:
### 설정값
#이미지를 따로 저장할 'img'라는 디렉토리를 미리 생성해주세요.

path = '/Users/pdh/TasteMaps/TasteMap/crawler/chromedriver' #크롬드라이버 위치

ID = 'wnwjq462@yonsei.ac.kr' #인스타 id
PW = 'xhfl798!!798' #인스타 비밀번호

SCROLL_NUM = 7 #얼마나 스크롤할 지 정하기(스크롤을 내려 게시물을 많이 확보해야 합니다. 값이 최소 1이어야 합니다.)
CRAWLING_NUM = 30 #크롤링할 게시물 개수


tag_list = dining_list[450:500] #검색할 range 입력
tag_list

,dining_id,new_dining_name,total_name,dining_name,branch_name,addr
450,451,신촌요릿집,요릿집,요릿집,NaN,\n서울 서대문구 연희맛로 36\n(지번) 연희동 137-4\n
451,452,신촌오렌지,오렌지 신촌점,오렌지,신촌점,\n서울 서대문구 명물길 37\n(지번) 창천동 45-1\n
452,453,신촌잘들어,잘들어,잘들어,NaN,\n서울 서대문구 연희맛로 7 1층\n(지번) 연희동 189-11\n
453,454,신촌와플대학 연희캠퍼스,와플대학 연희캠퍼스,와플대학 연희캠퍼스,NaN,\n서울 서대문구 연희로 90 1층\n(지번) 연희동 187-60\n
454,455,신촌예티,예티 신촌점,예티,신촌점,\n서울 서대문구 연세로12길 23 2층\n(지번) 창천동 2-2\n
455,456,신촌포차천국,포차천국 신촌점,포차천국,신촌점,\n서울 서대문구 연세로7안길 19\n(지번) 창천동 52-63\n
456,457,신촌피자조팩토리,피자조팩토리,피자조팩토리,NaN,\n서울 서대문구 연세로11길 5 3층\n(지번) 창천동 33-5\n
457,458,신촌유야케도쿄,유야케도쿄,유야케도쿄,NaN,\n서울 서대문구 이화여대3길 28 1층 101호\n(지번) 대현동 37-31\n
458,459,신촌아마,아마,아마,NaN,\n서울 서대문구 연세로7길 24 2층\n(지번) 창천동 52-67\n
459,460,신촌두끼떡볶이,두끼떡볶이 이대점,두끼떡볶이,이대점,\n서울 서대문구 이화여대길 89 2층\n(지번) 대현동 27-7\n


In [7]:
def get_url(driver, tag):
    url = "https://www.instagram.com/explore/tags/"
    url = url + quote_plus(tag['new_dining_name'])
    driver.get(url)
    time.sleep(4)

    html = driver.page_source
    soup = bs(html, 'lxml')

In [8]:
def scroll(drive, scroll_num):
    #컨텐츠가 없으면 False 반환 -> 다음 태그로 넘어감
    try :  
        driver.find_element_by_class_name('_9AhH0')
    except :
        print('#################No Content##################')
        return False
    #SCROLL_NUM만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
    for _ in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        
    driver.execute_script('window.scrollTo(0,0);')
    #첫 번째 게시물
    
    driver.find_element_by_class_name('_9AhH0').click()
    return True

In [9]:
def crawl(driver, crawl_num, tag):
    
    n = 0
    
    for i in range(crawl_num):
        html = driver.page_source
        soup = bs(html, 'lxml')
        time.sleep(2)

        contents = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
        if len(contents) == 0:
            contents = np.nan
        else:
            contents = contents[0].text
            contents = unicodedata.normalize('NFC', contents)
            
        print(contents)
        dining_ids.append(tag['dining_id'])
        total_names.append(tag['total_name'])
        sources.append('인스타그램')
        contents_list.append(contents)

        #다음 게시물 , 없으면 종료
        try : 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(3)
        except :
            return
        
        n += 1

In [13]:
### 실행코드

dining_ids = []
total_names = []
sources = []
rate = []
contents_list = []
start = time.time()

driver = webdriver.Chrome(path)
driver.maximize_window()
driver.get('https://instagram.com/explore')
time.sleep(1)

driver.find_element_by_name('username').send_keys(ID)
driver.find_element_by_name('password').send_keys(PW)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(3)

for index,tag in tag_list.iterrows():
    print(tag['new_dining_name'])
    get_url(driver, tag)
    time.sleep(3)
    
    if not scroll(driver, SCROLL_NUM) :
        continue
    crawl(driver, CRAWLING_NUM, tag)
    
    
print("총 걸린 시간 :", time.time() - start)
'''
### 저장
data = {'dining_id':dining_ids,
        'total_name':total_names,
         'review':contents_list,
         'source': sources}
    
df = pd.DataFrame(data)
df.to_csv("test_data.csv", index=True, encoding='utf-8-sig')
'''
### 저장
data = {'dining_id':dining_ids,
        'total_name':total_names,
         'review':contents_list,
         'source': sources}
    
df = pd.DataFrame(data)
df.to_csv("reviews_instagram_450-500.csv", index=False, encoding='utf-8-sig')

신촌요릿집
#################No Content##################
신촌오렌지
#신촌염색 #신촌오렌지 #신촌남자염색 #신촌미용실 #신촌엘세븐헤어 #엘세븐헤어 #염색 #남자염색
신촌잘들어
#################No Content##################
신촌와플대학 연희캠퍼스
#################No Content##################
신촌예티
예티 신촌점도 꿀맛!! 두번째 방문

#신촌예티 #인도커리 #NepaliCurry #신촌 #예티 #신촌맛집 #신촌인도요리 #신촌커리 #예티투
[English Below]제대로된 정통 인도/네팔식 커리 #홍대예티 🍛 한국에서 맛볼 수 있는 최고의 할랄 인증 인도 음식점 중 하나죠. 그린샐러드, 커리, 탄두리치킨, 난, 밥 그리고 음료까지 포함된 런치세트야 말로 가성비 갑! 지금 셔틀딜리버리에서 주문해보세요. @yeti2_indian_food

You don’t have to search the snowy Himalayas to find Yeti in Hongdae, they are conveniently listed on Shuttle! Not your average Indian joint, Yeti serves up some top notch Indian and Nepalese cuisine. Choose from all the Indian classics, but don’t miss the lunch sets, which deliver excellent value!

#신촌예티 #신촌배달맛집 #신촌맛집 #이대맛집 #이대배달맛집 #셔틀딜리버리 #셔틀 #배달음식 #Shuttle
인도요리는 정말 너무 맛있다ㅠ
사진찍을틈도없이 다먹어버려서 추가로 주문한 난만 찍었당...
크리스피&쫄깃한 난 먹을때마다 눈돌아가서 숨도안쉬구 두장쯤은 흡입해줘야함
아 넘나 행복ㅠㅠ♡♡♡ #인도요리 #신촌맛집 #신촌인도요리 #신촌예티 #예티 #예티2 #신촌데이트 #신촌 #난 #커리
인도갔

❤#신촌포차천국 ❤ ⠀
맛있는 술상을 집에서도 받아보세요💕
#배달 🛵
⠀

집밖은위험해 ⠀
⠀
⠀
배달지역 : #서대문#마포구 일대
⠀
#배달의민족 메뉴,상호검색--->>#포차천국 ⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
#신촌안주맛집
#안주스타그램
#닭볶음탕
#닭발
#양념치킨
#치킨
#두부김치
#골뱅이무침
#안주맛집
#떡볶이
#야식
#배달맛집
#야식배달
#야식추천
#홍대
#이대
#신촌술집 ⠀
#신촌배달
#신촌역맛집
#신촌역술집
#신촌역 ⠀
#신촌
#술스타그램
#배고파 ⠀
⠀
⠀
⠀
⠀
⠀
⠀ ⠀
⠀ ⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀ ⠀
#신촌 #벚꽃 #생맥주 💕
⠀
⠀
⠀
⠀
⠀
봄날에서 벌써 여름?
신촌 벚꽃보면서 생맥한잔~🍺
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
⠀
#신촌포차천국
#신촌맛집
#신촌술집
#신촌역맛집
#얼음맥주
#포차천국
⠀
⠀
⠀
#아빠찌개 #흐린날 #안주최고
⠀
⠀
⠀
⠀
⠀
⠀

무슨 맛?
빨간 맛
짬뽕 매운 맛~
얼큰한
아빠들이 좋아하는 소주안주
탕수육(꿔바로우&면사리)

⠀
⠀
#신촌포차천국
#불금
#안주스타그램
#포차천국
#안주맛집
#포차천국신촌점
#신촌
#shinchon
#신촌포차천국
포차천국에서안주뿌시기ㅋㅋㅋㅋ
식사부터안주까지해결!!
안먹어본메뉴위주로👍👍
밝을때부터캄캄해질때까지🤣🤣
꿀막걸리꿀떡꿀떡먹고힘들...🤪🤪
.
.
#포차천국신촌점 #포차천국 #안주뿌시기 #식사해결 #안주해결 #먹스타그램 #먹스타 #맛스타그램 #맛스타 #인스타푸드 #푸드스타그램 #먹부림 #먹방 #안주스타그램 #안주그램 #안주 #배터짐주의 #숙취주의 #헤롱헤롱 #맥주마무리 #꿀막걸리 #막걸리 #치킨 #막창 #순두부찌개 #쥐포 #콘치즈 #칼로리폭발 #기나긴1차
#신촌포차천국
정말오랜만에방문ㅎㅎ
배부른상태로가서먹태랑내사랑콘치즈😍😍
역시맛있음♡
테라.진로.참이슬주문하면홀스도줌ㅋㅋㅋ
혹시나하고사장님께인사했는데
기억하고계셔서완전깜놀ㅋㅋㅋ
작년여름에놓고간구두도보관해주시고~
사장님최고👍👍
.
@_pocha_heaven_
사장님앞으로자주갈게용ㅎ
구두챙겨주셔서감사합니다😆

인도요리🥸
#신촌아마
오랫만에 간 교류회 마치고 인도요리 맛집을 처음 가봤는데 생각보다 입에 잘 맞더군요 ^^
화덕 불맛이 일품인 탄두리치킨~~
커리에 찍어먹는 난~~
가끔은 한식 일식 중식 외의 이국적인 요리 경험도 좋은듯 하네요 ^^
#카케하시교류회 #신촌아마 #신촌인도요리전문점
가끔 생각나는 향신료 가득 인도커리🙏
#2020년7월31일 #인도요리#신촌아마

#치킨마크니 랑 나머지 커리는 머 시켰는지 기억이 안남..3개월 전인데 말이죠..

간만에 인도음식 먹어서 좋았고 분위기도 화려해서 해외나온듯 이국적인 느낌 물씬~
201013
들어오자마자 인도 온 줄,,,
一進店裡還以為我來到了印度
#신촌맛집 #인도요리 #인도요리맛집 #커리맛집 #신촌아마
카레는 좋아하지 않지만 커리는 좋아해💕
신라면 매운맛이라고 했는데 너무 매워서 콧물 줄줄😂 근데 너무 맛있게 매워서 멈출 수가 없다... 내일 힘들걸 알지만 마지막 한입까지🤤
손바닥 보다 훠어어어얼씬 큰 난까지 완벽쓰. 역시 현지인이 해서 다르군🥰
.
.
.
#커리#인도커리#신촌맛집#신촌커리#프라운빈달루#치킨마크니#아마#신촌아마#일상#먹방#먹스타그램#먹팔#먹팔해요#선팔#맞팔#좋아요#좋반#좋아요반사#좋테#첫줄반사#daily#mukbang#like#like4likes#likeforlikes#follow4followback#followforfollowback#f4f#foodie#amma
#일상 #친구와함께 #스타필드고양 #에베레스트 #인도카레 #평타 #그냥저냥 #역시 #인도카레는 #신촌아마
인도요리 아마
카레는 걍 그런데 커리는 좋아함.

버터갈릭난,치즈난이 일품.

탄두리치킨은 노~

#인도요리#신촌아마#아마#신촌인도요리#난#커리#탄두리치킨
카레에에~~
이제 담달에 먹을 수 있는 카레에에에
갈릭버터난 빨리 또 먹고싶따아아
#카레에빠진아이들 #신촌아마
⠀
샘과 프로도의 능지탈출 대작전
⠀
⠀
⠀
⠀
#방탈출 #룸익스케이프 #호빗 #음식 #인도요리 #신촌아마 #food
1인 1난은 #인도커리 먹을 때 필수지~
바로 구운 #버

#################No Content##################
신촌리틀파파포
#################No Content##################
신촌루프탑캠핑
비버지수🌿🌿🌿🌿

#신촌맛집#먹스타그램맞팔#먹스타그램#치맥#치킨#루프탑캠핑#신촌루프탑캠핑#신촌박스퀘어#신촌치맥#신촌캠핑#신촌테라스#신촌테라스술집#신촌테라스맛집#이대맛집#이대술집#이대박스퀘어#이대치맥
𝚁𝚘𝚘𝚏𝚝𝚘𝚙 ☾
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
#00 #01 #일상 #일상그램 #일상스타그램 #데일리 #데일리그램 #데일리스타그램 #daily #아이폰se2 #좋아요 #셀피 #selfie #selfi #셀스타그램 #신촌 #신촌루프탑카페 #신촌루프탑캠핑 #루프탑캠핑 #예쁜카페 #신촌카페 #예쁜하늘
인문경매1,2,3기 단체모임
스터디도 삶에 대한 이야기도 여전히 성장중인분들
오랜만에 너무나 반가웠습니다♥

#신촌루프탑캠핑 사장님 넘나 귀여우심
#인문경매 #스터디모임
#이대루프탑 #분위기깡패
#사장님완전친절
나 파인애플피자 진짜 사랑하는데 여기가 젤 맛있다 다른곳은 뭔 콤비네이션에 파인애플 몇쪼가리 얹어놓구 파인애플피자라 우기는데 여기는 치즈도 파인애플 맛이랑 어울리게 잘 써서 진짜 맛있음!!!!!!
#신촌 #신촌맛집 #신촌역 #루프탑캠핑 #신촌루프탑캠핑 #피자 #피자맛집 #파인애플피자 #🍕 #맥주내꺼아님 #ㅡㅡ
남편님과둘이추석연휴마무리!💑
내일부터다시열일모드!!🤔👏
요거다~~마셔버릴거야!!🍺🍴🍕
#우리에게는마지막연휴#우리에게연휴가있었나?#연휴내내일#오늘그래도좋았다 #남편고마움#남편그만좀삐지기♡#신촌루프탑캠핑#신촌기차역루프탑캠핑
#ROOFTOPCAMPING#치즈화덕피자강추#남편술안먹고피자만먹기있기없기?#포테이토9천원#맛스타그램#팔로우미#일상그램
신촌돈까스가땡길때
#################No Content##################
신촌빙달
#################No Content##################
신촌연희라운지
#####

In [76]:
### 저장
data = {'dining_id':dining_ids,
        'total_name':total_names,
         'review':contents_list,
         'source': sources}
    
df = pd.DataFrame(data)
#df.to_csv("reviews_instagram.csv", index=False, encoding='utf-8-sig')

In [11]:
df

,dining_id,total_name,review,source
0,403,굽네치킨 서대문연희점,"💡 오늘은 배달 닭 특집 (2) 🍗구운 치킨땡길때,,, 구웠는데 약간의 바삭함이 땡...",인스타그램
1,404,티원 연세대점,고급스러움 흐르는 중식당🥢\n신촌에 있는 티원🤭\n새우가지가 그렇게 맛있데🙋‍♂️\...,인스타그램
2,404,티원 연세대점,매운맛에 이끌림 🤩\n#후난식매운새우고추 ➡️ 중식당 티원 연세대점\n\n#신촌맛집...,인스타그램
3,404,티원 연세대점,솔직히 너무 깊은거 아냐?🤭\n새우가지 . 미쳤음 👍\n➡️ 티원 연세대점\n#신촌...,인스타그램
4,404,티원 연세대점,2018년 경사대 스경과\n\n#신년회 #경희사이버 #사이버대학교 #스포츠경영학과 ...,인스타그램
...,...,...,...,...
458,443,본죽 연대대신동점,試試新口味～芥菜牡蠣粥Oyster Porridge😋，一開始看不大出來，其實牡蠣肉蠻多的說...,인스타그램
459,447,동대문엽기떡볶이 신촌점,#동대문엽기떡볶이 #맵기조절가능 #땡초\n#덜매운맛 #신촌맛집 #신촌동대문엽기떡볶이...,인스타그램
460,450,웅네서서갈비,[신촌] 웅네서서갈비\n.\n.\n✔️ 넓고 쾌적한 내부\n✔️ 통풍이 잘되는 자연...,인스타그램
461,450,웅네서서갈비,결혼13주년\n고기좋아하는 우리부부~^^\n신촌 웅네서서갈비에 소맥!\n고기사진은 ...,인스타그램


In [3]:
df = pd.read_csv("reviews_instagram_0-100.csv")

In [29]:
tf = pd.read_csv("reviews_instagram_450-500.csv")

In [30]:
tf

,dining_id,total_name,review,source
0,452,오렌지 신촌점,#신촌염색 #신촌오렌지 #신촌남자염색 #신촌미용실 #신촌엘세븐헤어 #엘세븐헤어 #염...,인스타그램
1,455,예티 신촌점,예티 신촌점도 꿀맛!! 두번째 방문\n\n#신촌예티 #인도커리 #NepaliCurr...,인스타그램
2,455,예티 신촌점,[English Below]제대로된 정통 인도/네팔식 커리 #홍대예티 🍛 한국에서 ...,인스타그램
3,455,예티 신촌점,인도요리는 정말 너무 맛있다ㅠ\n사진찍을틈도없이 다먹어버려서 추가로 주문한 난만 찍...,인스타그램
4,455,예티 신촌점,인도갔다왔다.\n\n계절학기-병원-모의고사. 밀도가 너무 빡센 극한하루였따😧\n\n...,인스타그램
...,...,...,...,...
217,476,포티드,다시봐도 이날 분위기 넘 맘에든당💕\n요기 옆에 타로집 되게 잘맞아서\n타로보고 커...,인스타그램
218,476,포티드,헤리포터 카페\n.\n.완전 음악까지 헤리포터 음악나오고\n영화속에 들어온거 같음 ...,인스타그램
219,476,포티드,ෆ ෆ ෆ\n\n신촌 예쁜 카페 포티드✨\n드디어 가봤는데 예쁘다 :)\n.\n.\...,인스타그램
220,476,포티드,📍신촌 포티드\n카페 앞에서 사진 꼭찍어야하는 포토스팟📸 연인이랑 손잡고 고고띵💑\...,인스타그램


In [7]:
new_df = df.append(tf)

In [31]:
new_df = new_df.append(tf)

In [32]:
new_df

,dining_id,total_name,review,source
0,3,콘하스 연희점,오랜만에 대딩친구들 다 만났다\n오랜만에 만난 지영이랑 원래 잘 만나는 지은이민정이...,인스타그램
1,4,연경 연희점,.\n오늘은 신촌,인스타그램
2,4,연경 연희점,#신촌 에서 토요일날 #마마무 도 보고 개이득!\n근데 멀리서 줌 땡겨가지고 화질은...,인스타그램
3,4,연경 연희점,"#보통술집 #마지막\n하루키, 원부술집에 이어 이렇게 또 이별을\n전, 현직원 거의...",인스타그램
4,4,연경 연희점,얼마부터 배달되냐 여쭤봤더니 동네니까 해드릴게여 라며 ㅇㅋ 해주신 사장님 감사해요\...,인스타그램
...,...,...,...,...
217,476,포티드,다시봐도 이날 분위기 넘 맘에든당💕\n요기 옆에 타로집 되게 잘맞아서\n타로보고 커...,인스타그램
218,476,포티드,헤리포터 카페\n.\n.완전 음악까지 헤리포터 음악나오고\n영화속에 들어온거 같음 ...,인스타그램
219,476,포티드,ෆ ෆ ෆ\n\n신촌 예쁜 카페 포티드✨\n드디어 가봤는데 예쁘다 :)\n.\n.\...,인스타그램
220,476,포티드,📍신촌 포티드\n카페 앞에서 사진 꼭찍어야하는 포토스팟📸 연인이랑 손잡고 고고띵💑\...,인스타그램


In [34]:
new_df.isnull().sum()

dining_id       0
total_name      0
review        357
source          0
dtype: int64

In [35]:
new_tf = new_df.dropna()

In [38]:
new_tf.to_csv("reviews_instagram_total.csv", index=False, encoding="utf-8-sig")